# Multiple Stratigies For Audio

## Creating your tourch audio dataset 

In [ ]:
print("test")

In [7]:
%pip install pandas
%pip install torch==2.7.0 torchaudio==2.7.0


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for torch==2.7.0 from https://files.pythonhosted.org/packages/9c/58/2d245b6f1ef61cf11dfc4aceeaacbb40fea706ccebac3f863890c720ab73/torch-2.7.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for torchaudio==2.7.0 from https://files.pythonhosted.org/packages/78/98/ec8c7aba67b44cdc59717d4b43d02023ded5da180d33c6469d20bf5bfa3c/torchaudio-2.7.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/4d/36/2a115987e2d8c300a974597416d9de88f2444426de9571f4b59b2cca3acc/filelock-3.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for sympy>=1.13.3 from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e5681d0a6c10b75bf35619

In [9]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import soundfile as sf
import torch
import torchaudio
from torch.utils.data import Dataset
import os
from pathlib import Path

Example of dataset class

In [ ]:
class m_audio_dataset(Dataset):

    def __init__(self,
                    audio_dir,
                    transformation,
                    num_samples,
                    pd_data,
                    kind='train'):
        self.audio_dir = audio_dir
        self.num_samples = num_samples
        if (transformation):
            self.transformation = transformation

        if kind=='train':
            self.items = pd_data[pd_data['train'] == True][['recordedName','label','className']]
            self.items.reset_index(inplace = True,drop = True)
        elif kind=='test':
            self.items = pd_data[pd_data['train'] == False][['recordedName','label','className']]
            self.items.reset_index(inplace = True,drop = True)

        else:
            self.items =  pd_data[['recordedName','label','className']]
            self.items.reset_index(inplace = True,drop = True)

        self.length = len(self.items)

        self.targets = torch.as_tensor( list(self.items['label']))
    def __len__(self):
        return self.length


    def __getitem__(self, index):
        filename = self.items['recordedName'][index]
        className = self.items['className'][index]
        label = self.items['label'][index]
        signal, rate = torchaudio.load(self.audio_dir+ className + '/' + filename)
        signal = self._to_mono_if_necessary(signal)
        signal = self._cutting_if_necessary(signal)
        signal = self._padding_if_necessary(signal)
        data_tensor = self.transformation(signal)
        return (data_tensor, int(label))


    def _to_mono_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _cutting_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal

    def _padding_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [ ]:
def loadDataSet(setType="train",
                ballance=False,
                FRAME_SIZE = 1024,
                HOP_LENGTH = 512,
                N_MELS = 60):

    BASE_DIR = ".."
    ANNOTATIONS_FILE = BASE_DIR + "/recordingInfo/validRecordings.csv"
    AUDIO_DIR = BASE_DIR + "/recordedAudio/"

    pd_data = pd.read_csv(ANNOTATIONS_FILE, index_col=0)
    # adding label column
    ord_enc = OrdinalEncoder()
    pd_data["label"] = ord_enc.fit_transform(pd_data[["className"]]).astype(int)

    # splittint train test
    pd_data['train'] = False
    trainPercent = 0.7
    for labelVal in pd_data.label.unique():
        trainCount = int(len(pd_data[pd_data.label == labelVal]) * trainPercent)
        pd_data.loc[pd_data[pd_data.label == labelVal].head(trainCount).index, 'train'] = True
    file = pd_data['recordedName'][0]
    className = pd_data['className'][0]
    fullName = AUDIO_DIR + className + '/' + file
    data_array, samplerate = sf.read(fullName)


    if(ballance):
        minClassSize = min([len(pd_data[pd_data['label'] == 0]), len(pd_data[pd_data['label'] == 1]),
                            len(pd_data[pd_data['label'] == 2]), len(pd_data[pd_data['label'] == 3])])
        for i in range(4):
            pd_data = pd_data.drop(pd_data[pd_data['label'] == i].index[minClassSize:])

        pd_data = pd_data.reset_index(drop=True)
        # splittint train test
        pd_data['train'] = False
        trainPercent = 0.7
        for labelVal in pd_data.label.unique():
            trainCount = int(len(pd_data[pd_data.label == labelVal]) * trainPercent)
            pd_data.loc[pd_data[pd_data.label == labelVal].head(trainCount).index, 'train'] = True


    SAMPLES_SIZE = int(5.8 * samplerate)
    SAMPLE_RATE = samplerate

    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=FRAME_SIZE,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS,
        normalized=True
    )

    return m_audio_dataset(AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLES_SIZE,
                            pd_data,
                            kind=setType) , 4

In [13]:
config={}
config['FRAME_SIZE'] = 1024
config['HOP_LENGTH'] = 512
config['N_MELS'] = 60

config['lr_base'] = 0.1
config['wght_decay'] = 1E-05
config['momentum'] = 0.9
config['memory_size'] = 2000

config['batch_size'] = 32
config['nb_exp'] = 4
config['epochs'] = 2
config['lr_milestones_1'] = 49
config['lr_milestones_2'] = 63
config['lr_factor'] = 5
config['seed'] = 1234
config['modelType'] = 'icarlNet'
config['model_hs'] = 1000
config['opt'] = 'ADAM'



In [ ]:
train_data, nb_exp = loadDataSet("train",False,config['FRAME_SIZE'] ,config['HOP_LENGTH'] ,config['N_MELS'])
test_data, nb_exp = loadDataSet("test",False,config['FRAME_SIZE'] ,config['HOP_LENGTH'] ,config['N_MELS'])


In [14]:
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training import GDumb , ICaRL
from avalanche.models import IcarlNet, make_icarl_net, initialize_icarl_net
from avalanche.models import SimpleMLP, as_multitask
from torch.optim import SGD , Adam
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import (
    ExperienceAccuracy,
    StreamAccuracy,
    EpochAccuracy,
)
from avalanche.logging.interactive_logging import InteractiveLogger
from torch.optim.lr_scheduler import MultiStepLR
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
import time
import pandas as pd
from torchvision import transforms

ModuleNotFoundError: No module named 'avalanche'

In [11]:
train_data_avalanche = AvalancheDataset(train_data)
test_data_avalanche = AvalancheDataset(test_data)

NameError: name 'AvalancheDataset' is not defined

In [ ]:
###  The benchmark
### ### ### ### ### ### ### ### ### ### ### ### ### ###
scenario = nc_benchmark(
        train_dataset=train_data,
        test_dataset=test_data,
        n_experiences=config["nb_exp"],
        task_labels=False,
        seed=config['seed'],
        shuffle=False,
    )


In [15]:
def transformData(inp):
    return inp

In [ ]:
### Evaluation , plugins, and loggers
### ### ### ### ### ### ### ### ### ### ### ### ###

evaluator = EvaluationPlugin(
        EpochAccuracy(),
        ExperienceAccuracy(),
        StreamAccuracy(),
        loggers=[InteractiveLogger()],
    )

In [ ]:
### The Model
### ### ### ### ### ### ### ### ### ### ### ### ###

if(config['modelType'] == "icarlNet"):
    model = make_icarl_net(num_classes=config['nb_exp'],n=5, c=1)
    model.apply(initialize_icarl_net)
else:

    model = SimpleMLP(num_classes=config['nb_exp'],
                        input_size=data_tensor.size()[1] * data_tensor.size()[2],
                        hidden_size=config['model_hs'] ,
                        drop_rate=0)


if (config['opt' ]== "ADAM"):
    optim = Adam(
        model.parameters(),
        lr=config['lr_base'],
        weight_decay=config['wght_decay'],
    )



else:
    optim = SGD(
        model.parameters(),
        lr=config['lr_base'],
        weight_decay=config['wght_decay'],
        momentum= config['momentum'],
    )

sched = LRSchedulerPlugin(
    MultiStepLR(optim, [config['lr_milestones_1'],config['lr_milestones_2']], gamma=1.0 / config['lr_factor'])
)

In [ ]:
### the strategy
### ### ### ### ### ### ### ### ### ### ### ### ### ###


strategy = ICaRL(
    model.feature_extractor,
    model.classifier,
    optim,
    config['memory_size'],
    buffer_transform=transforms.Compose([transformData]),
    device=device,
    fixed_memory=True,
    train_mb_size=config['batch_size'],
    train_epochs=config['epochs'],
    eval_mb_size=config['batch_size'],
    plugins=[sched],
    evaluator=evaluator,
)

In [ ]:
### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
###  Training Code
print("Starting experiment...")
evaluationResults = []
processingTime = []
startingTime = time.time()
print("startingTime : ",startingTime)
for  i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i +1 ]

    print("Start training on experience ", exp.current_experience)
    expTime = time.time()
    print("starting a new exp time : ",expTime)
    val_exps = [e for e in scenario.test_stream][: i +1 ]
    strategy.train(exp, num_workers=4)
    print("End training on experience", exp.current_experience)

    evaluationResults.append(strategy.eval(eval_exps, num_workers=4 ))
    print("Computing accuracy on the test set")
    expEndTime = time.time()
    expDuration = expEndTime - expTime
    processingTime.append(expDuration)

endTime = time.time()
print("endTime : ",endTime)
print("total Processing time :" ,endTime-startingTime)

procTime = endTime-startingTime


num_classes = config['nb_exp']
evaluationDfColumns = ['Exp', 'testAcc', 'trainAcc']
lst = range(0, num_classes)
evaluationDfColumns = evaluationDfColumns + ["testAccExp{:d}".format(x) for x in lst]

evaluationDf = pd.DataFrame(columns=evaluationDfColumns)
trainingDf = pd.DataFrame(columns=evaluationDfColumns)

for i in range(0, num_classes):

    evaluationLine = {}
    evaluationLine = {'Exp': i,
                        'testAcc': evaluationResults[i]['Top1_Acc_Stream/eval_phase/test_stream/Task000'],
                        'trainAcc': evaluationResults[i]['Top1_Acc_Epoch/train_phase/train_stream/Task000']}

    for j in range(0, num_classes):
        if (j <= i):
            evaluationLine["testAccExp{:d}".format(j)] = evaluationResults[i][
                "Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{:03d}".format(j)]

    new_row = pd.Series(data=evaluationLine)

    evaluationDf = evaluationDf.append(new_row, ignore_index=True)

evaluationDf.to_csv(BaseSaveDir + "ICARL_{:d}.csv".format(runningIdx), index=False)


print("Ending the  seed ", config["seed"] )
print("last set training acc:",evaluationDf['trainAcc'].iloc[-1])
print("training acc:",evaluationDf['trainAcc'].mean())
print("testing acc:",evaluationDf['testAcc'].iloc[-1])

config['trainAccMean'] = evaluationDf['trainAcc'].mean()
config['trainAccLast'] = evaluationDf['trainAcc'].iloc[-1]
config['testAcc'] = evaluationDf['testAcc'].iloc[-1]
config['procTime'] = procTime
configDF = pd.DataFrame(config, index=[0])
configDF.to_csv(BaseSaveDir + "ICARL_Config_{:d}.csv".format(runningIdx), index=False)
